In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout, GlobalAveragePooling1D, AveragePooling1D
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import Dropout, BatchNormalization
import os
import random
import tensorflow as tf

In [2]:
# resetting the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

# import data
df = pd.read_csv('Boruta_onchain_data.csv')
df1 = pd.read_csv('all_data.csv')

df = df[df['timestamp'] >= '2013-03-11'].reset_index(drop=True)

#onchain_data_new
#TA_data
#all_data
#Boruta_data
#Boruta_TA_data
#Boruta_onchain_data

In [3]:
# separate the inputs and target
X = df.drop('timestamp', axis=1)

# create binary classification for price movement. this assigns 1 to y if price will move upward next day.
price = pd.DataFrame()
price['today'] = df1['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = price['next day']

# Drop the last row where 'next day' would be NaN after shifting
X = X[:-1]
y = y.dropna()

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 5
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

In [5]:
# define the CNN-GRU model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, X_train_reshaped.shape[2])))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=1))
model.add(Dropout(0.5))
model.add(GRU(units=128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(GRU(units=80))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error', metrics=['mean_absolute_error', 'mean_squared_error'])

C:\Users\oluwa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.save("gru_r_model.h5")

In [6]:
# train with the training dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=50, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
54/54 [==============================] - 4s 20ms/step - loss: 3467.4851 - mean_absolute_error: 3467.4851 - mean_squared_error: 27462036.0000 - val_loss: 31422.1328 - val_mean_absolute_error: 31422.1328 - val_mean_squared_error: 1320651392.0000
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 3466.7363 - mean_absolute_error: 3466.7363 - mean_squared_error: 27453898.0000 - val_loss: 31421.0234 - val_mean_absolute_error: 31421.0234 - val_mean_squared_error: 1320578048.0000
Epoch 3/100
54/54 [==============================] - 0s 7ms/step - loss: 3465.4944 - mean_absolute_error: 3465.4944 - mean_squared_error: 27429646.0000 - val_loss: 31417.6777 - val_mean_absolute_error: 31417.6777 - val_mean_squared_error: 1320352256.0000
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: 3463.4204 - mean_absolute_error: 3463.4204 - mean_squared_error: 27414262.0000 - val_loss: 31416.1133 - val_mean_absolute_error: 31416.1133 - val_mean_squared_er

In [7]:
# predict with testing dataset
y_pred = model.predict(X_test_reshaped)

24/24 [==============================] - 1s 3ms/step


In [8]:
# evaluate the prediction performance
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Root Mean Squared Error: 28780.126975781033
Mean Absolute Error: 25780.27932043554
R^2 Score: -4.073988224762178


In [9]:
time = df['timestamp']
time_train, time_test = train_test_split(time, test_size=0.2, shuffle=False)
time_test = time_test[timesteps:]

# Flatten y_pred to be a 1-dimensional array
y_pred_flat = y_pred.flatten()

# Create a DataFrame with columns time_test, y_test, and y_pred
pred_res = pd.DataFrame({'date': time_test, 'actual': y_test.values, 'prediction': y_pred_flat, 'value': price['today'][-748:]})

pred_res.to_csv('pred2/gru_uni_data.csv', index=False)